# Stable Diffusion Interactive Notebook 📓 🤖

A widgets-based interactive notebook for Google Colab that lets users generate AI images from prompts (Text2Image) using [Stable Diffusion (by Stability AI, Runway & CompVis)](https://en.wikipedia.org/wiki/Stable_Diffusion).

This notebook aims to be an alternative to WebUIs while offering a simple and lightweight GUI for anyone to get started with Stable Diffusion.

Uses Stable Diffusion, [HuggingFace](https://huggingface.co/) Diffusers and [Jupyter widgets](https://github.com/jupyter-widgets/ipywidgets).

<br/>

Made with ❤️ by Nemo1166, origin by [redromnon](https://github.com/redromnon/stable-diffusion-interactive-notebook)

To-do:
- Model settings
    - [x] Add more sampler (scheduler)
    - [x] Add support for loading model from single file (\*.ckpt / \*.safetensors)
    - [ ] Add VAE selection
    - [ ] Add support for loading LoRA(s)
- Text2image
    - [x] Add CLIP skip
- Others
    - [ ] Add Hires.fix (txt2img -> upscale -> img2img)
    - [ ] Colab form-format input

In [ ]:
#@title 👇 Installing dependencies { display-mode: "form" }
#@markdown ---
#@markdown Make sure to select **GPU** as the runtime type:<br/>
#@markdown *Runtime->Change Runtime Type->Under Hardware accelerator, select GPU*
#@markdown
#@markdown ---

%pip -q install diffusers transformers accelerate scipy safetensors xformers mediapy ipywidgets==8.1.1 omegaconf

In [ ]:
#@title 👇 Selecting Model { form-width: "20%", display-mode: "form" }
#@markdown ---
#@markdown - **Select Model** - A list of Stable Diffusion models to choose from.
#@markdown  - **Link** - Load model from Stable-diffusion single file (support .ckpt or .safetensors).
#@markdown  - **Selector** - Choose a pre-trained diffusers model.
#@markdown - **Safety Checker** - Enable/Disable uncensored content
#@markdown
#@markdown ---

from diffusers import StableDiffusionPipeline
from diffusers.models import AutoencoderKL
import torch
import ipywidgets as widgets
import importlib

#Enable third party widget support
from google.colab import output
output.enable_custom_widget_manager()

#Pipe
pipe = None

# Model source selector
model_src_switcher = widgets.Button(
    description="Switch model source",
    button_style="info"
)

# Single model file src
model_src = widgets.Textarea(
    value="https://huggingface.co/nemo1166/cetusmix-wf2/blob/main/cetusMix_Whalefall2.safetensors",
    placeholder="HTTP link to single file",
    description="Model link:",
    rows=1,
    layout=widgets.Layout(width="600px", visibility='hidden')
)

#Models
select_model = widgets.Dropdown(
    options=[
        ("Stable Diffusion 2.1 Base" , "stabilityai/stable-diffusion-2-1-base"),
        ("Stable Diffusion 2.1" , "stabilityai/stable-diffusion-2-1"),
        ("Stable Diffusion 1.5", "runwayml/stable-diffusion-v1-5"),
        ("Dreamlike Photoreal 2.0" , "dreamlike-art/dreamlike-photoreal-2.0"),
        ("OpenJourney v4" , "prompthero/openjourney-v4")
    ],
    description="Select Model:"
)

#Safety Checker
safety_check = widgets.Checkbox(
    value=True,
    description="Enable Safety Check",
    layout=widgets.Layout(margin="0px 0px 0px -85px")
)

#Output
out = widgets.Output()

#Apply Settings
apply_btn = widgets.Button(
    description="Apply",
    button_style="info"
)

# Switch model loading method
use_SFM = False
def switch_model_src(p):
    global use_SFM
    out.clear_output()

    with out:
        use_SFM = not use_SFM
        if use_SFM:
            select_model.layout.visibility = 'hidden'
            model_src.layout.visibility = 'visible'
            print("Mode: Loading model from single-file stable-diffusion model.")
        else:
            select_model.layout.visibility = 'visible'
            model_src.layout.visibility = 'hidden'
            print("Mode: Loading model from pre-trained diffusers.")

#Run pipeline
def pipeline(p):

  global pipe
  global use_SFM

  out.clear_output()
  apply_btn.disabled = True

  with out:

    print("Running, please wait...")

    if use_SFM:
        try:
            pipe = StableDiffusionPipeline.from_single_file(
                model_src.value,
                torch_dtype=torch.float16,
                vae=AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse", torch_dtype=torch.float16).to("cuda")
                ).to("cuda")
        except:
            raise Exception("Invalid model link.")
    else:
        pipe = StableDiffusionPipeline.from_pretrained(
            select_model.value,
            torch_dtype=torch.float16,
            vae=AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse", torch_dtype=torch.float16).to("cuda")
            ).to("cuda")

    if not safety_check.value:
      pipe.safety_checker = None

    # Optimization trickcs
    pipe.enable_xformers_memory_efficient_attention()
    pipe.enable_vae_tiling()

    print("Finished!")

  apply_btn.disabled = False


#Display
apply_btn.on_click(pipeline)
model_src_switcher.on_click(switch_model_src)

widgets.VBox(
    [
      widgets.HTML(value="<h2>Configure Pipeline</h2>"), model_src_switcher, model_src,
      select_model, safety_check, apply_btn, out
    ]
)


In [ ]:
#@title 👇 Model customisation { display-mode: "form" }
#@markdown ---
#@markdown - **Select Sampler** - A list of schedulers to choose from. Default is EulerAncestralScheduler.
#@markdown
#@markdown ---

#Schedulers
select_sampler = widgets.RadioButtons(
    options=[
        'Euler a', 'Euler', 'UniPC', 'DDIM', 'DDPM', 'PNDM', 'LMS',
        'DPM++2M Karras', 'DPM++2S', 'DPM2', 'DPM2 a',
        'Heun', 'UniPC',
    ],
    value='Euler a',
    description="Select Schedular:",
    orientation='horizontal',
    layout=widgets.Layout()
)
select_sampler.style.description_width = "auto"

# button
submit = widgets.Button(
    description="Submit",
    button_style="info"
)

#Output
customization = widgets.Output()

#Get scheduler
def set_scheduler(name):
    submit.disabled = True
    global pipe
    customization.clear_output()

    SCHEDULER_MAP ={
        'PNDM':'PNDMScheduler',
        'LMS':'LMSDiscretescheduler',
        'DDIM':'DDIMScheduler',
        'DDPM':'DDPMScheduler',
        'DPM++2M Karras':'DPMSolverMultistepScheduler',
        'DPM++2S':'DPMSolversinglestepscheduler',
        'DPM2':'KDPM2DiscreteScheduler',
        'DPM2 a':'KDPM2AncestralDiscretescheduler',
        'Euler':'EulerDiscreteScheduler',
        'Euler a':'EulerAncestralDiscreteScheduler',
        'Heun':'HeunDiscreteScheduler',
        'UniPC':'UniPCMultistepScheduler',
    }

    with customization:
        try:
            selected_scheduler = SCHEDULER_MAP[select_sampler.value]
            print(f"Using Sampler: {select_sampler.value} ({selected_scheduler}).")
            exec(f"from diffusers import {selected_scheduler}")
            pipe.scheduler = eval(f"{selected_scheduler}.from_config(pipe.scheduler.config)")
            print("Done.")
            submit.disabled = False
        except:
            raise Exception("Sampler unavailable.")

submit.on_click(set_scheduler)

widgets.VBox(
    [
      widgets.HTML(value="<h2>Customize Pipeline</h2>"),
      select_sampler, submit, customization
    ]
)

In [ ]:
#@title 👇 Generating Images { form-width: "20%", display-mode: "form" }
#@markdown ---
#@markdown - **Prompt** - Description of the image
#@markdown - **Negative Prompt** - Things you don't want to see or ignore in the image
#@markdown - **Steps** - Number of denoising steps. Higher steps may lead to better results but takes longer time to generate the image. Default is `30`.
#@markdown - **CFG** - Guidance scale ranging from `0` to `20`. Lower values allow the AI to be more creative and less strict at following the prompt. Default is `7.5`.
#@markdown - **Seed** - A random value that controls image generation. The same seed and prompt produce the same images. Set `-1` for using random seed values.
#@markdown ---
import ipywidgets as widgets, mediapy, random
import IPython.display


#PARAMETER WIDGETS
width = "300px"

prompt = widgets.Textarea(
    value="best quality, highly detailed, masterpiece, ultra-detailed, illustration",
    placeholder="Enter prompt",
    #description="Prompt:",
    rows=5,
    layout=widgets.Layout(width="600px")
)

neg_prompt = widgets.Textarea(
    value="lowres,bad anatomy,bad hands,text,error,missing fingers,extra digit,fewer digits,cropped,worst quality,low quality,normal quality,jpeg artifacts,signature,watermark,username,missing arms",
    placeholder="Enter negative prompt",
    #description="Negative Prompt:",
    rows=5,
    layout=widgets.Layout(width="600px")
)

num_images = widgets.IntText(
    value=1,
    description="Images:",
    layout=widgets.Layout(width=width),
)

clip_skip = widgets.IntSlider(
    value=2,
    min=0,
    max=11,
    step=1,
    description="CLIP skip:",
    orientation='horizontal',
    readout=True,
    readout_format='d',
    layout=widgets.Layout(width=width),
)

steps = widgets.IntText(
    value=20,
    description="Steps:",
    layout=widgets.Layout(width=width)
)

CFG = widgets.FloatText(
    value=7.5,
    description="CFG:",
    layout=widgets.Layout(width=width)
)

img_height = widgets.IntSlider(
    min=0,
    max=1024,
    step=8,
    value=512,
    description="Height:",
    orientation='horizontal',
    readout=True,
    readout_format='d',
    layout=widgets.Layout(width=width)
)

img_width = widgets.IntSlider(
    min=0,
    max=1024,
    step=8,
    value=512,
    description="Width:",
    orientation='horizontal',
    readout=True,
    readout_format='d',
    layout=widgets.Layout(width=width)
)

random_seed = widgets.IntText(
    value=-1,
    description="Seed:",
    layout=widgets.Layout(width=width),
    disabled=False
)

generate = widgets.Button(
    description="Generate",
    disabled=False,
    button_style="primary"
)

display_imgs = widgets.Output()


#RUN
def generate_img(i):

  #Clear output
  display_imgs.clear_output()
  generate.disabled = True

  #Calculate seed
  seed = random.randint(0, 2147483647) if random_seed.value == -1 else random_seed.value

  with display_imgs:

    print("Running...")

    images = pipe(
      prompt.value,
      height = img_height.value,
      width = img_width.value,
      num_inference_steps = steps.value,
      guidance_scale = CFG.value,
      num_images_per_prompt = num_images.value,
      negative_prompt = neg_prompt.value,
      generator = torch.Generator("cuda").manual_seed(seed),
      clip_skip = clip_skip.value,
    ).images
    mediapy.show_images(images)

    print(f"Seed:\n{seed}")

  generate.disabled = False

#Display
generate.on_click(generate_img)

widgets.VBox(
    [
      widgets.AppLayout(
        header=widgets.HTML(
            value="<h2>Stable Diffusion</h2>",
        ),
        left_sidebar=widgets.VBox(
            [num_images, steps, CFG, img_height, img_width, clip_skip, random_seed]
        ),
        center=widgets.VBox(
            [prompt, neg_prompt, generate]
        ),
        right_sidebar=None,
        footer=None
      ),
      display_imgs
    ]
)